In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("TRANSACTION_TAKEHOME.csv")
print("Dataset shape:", df.shape)

Dataset shape: (50000, 8)


In [3]:
df.head(5)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


## Initial Observations

- **Columns**: `RECEIPT_ID`, `PURCHASE_DATE`, `SCAN_DATE`, `STORE_NAME`, `USER_ID`, `BARCODE`, `FINAL_QUANTITY`, `FINAL_SALE`
- **Potential Issues**:
  - Some rows have a blank `FINAL_SALE`.
  - Some rows have `FINAL_QUANTITY` as `"zero"`.
  - The `BARCODE` field has missing values (`NaN`).

In [4]:
print("\nDataset info:")
df.info()


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECEIPT_ID      50000 non-null  object 
 1   PURCHASE_DATE   50000 non-null  object 
 2   SCAN_DATE       50000 non-null  object 
 3   STORE_NAME      50000 non-null  object 
 4   USER_ID         50000 non-null  object 
 5   BARCODE         44238 non-null  float64
 6   FINAL_QUANTITY  50000 non-null  object 
 7   FINAL_SALE      50000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.1+ MB


In [5]:
# Counting missing values in each column
missing_counts = df.isnull().sum()
print("Missing values per column:")
print(missing_counts)

# Count total rows
total_rows = len(df)
print("\nTotal number of rows:", total_rows)

Missing values per column:
RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64

Total number of rows: 50000


### ANALYZING MISSING BARCODES

In [6]:
# Group by STORE_NAME where BARCODE is null
missing_barcode_stores = (
    df[df["BARCODE"].isnull()]
    .groupby("STORE_NAME")["RECEIPT_ID"]
    .count()
    .reset_index(name="Count of Receipts")
    .sort_values(by="Count of Receipts", ascending=False)
)
missing_barcode_stores.head(5)

,STORE_NAME,Count of Receipts
3,ALDI,2338
45,CVS,606
53,DOLLAR TREE STORES INC,230
107,KROGER,228
200,WALGREENS,226


### Observation
- A significant portion of missing barcodes might be from specific stores (e.g., ALDI).
- This can affect brand-level or product-level analysis if not addressed.

In [7]:
# Checking if data is unique at RECEIPT_ID level
unique_receipt_ids = df["RECEIPT_ID"].nunique()
print("Number of unique RECEIPT_IDs:", unique_receipt_ids)
print("Total number of rows:", total_rows)

# Check uniqueness at (RECEIPT_ID, BARCODE) level
unique_rows = df.drop_duplicates(subset=["RECEIPT_ID", "BARCODE"])
unique_combos_count = len(unique_rows)
print("\nUnique (RECEIPT_ID, BARCODE) combos:", unique_combos_count)

# If unique_combos_count < total_rows, it means duplicates exist

Number of unique RECEIPT_IDs: 24440
Total number of rows: 50000

Unique (RECEIPT_ID, BARCODE) combos: 24795


In [8]:
# Checking unique values in FINAL_SALE
unique_final_sale = df["FINAL_SALE"].unique()
print("Unique values in FINAL_SALE:", unique_final_sale)

# Count how many rows have actually blank FINAL_SALE (null or empty string)
blank_final_sale_count = df["FINAL_SALE"].isnull().sum() + (df["FINAL_SALE"] == "").sum()
print("\nCount of blank FINAL_SALE values:", blank_final_sale_count)

Unique values in FINAL_SALE: [' ' '1.49' '3.49' ... '11.02' '20.17' '42.38']

Count of blank FINAL_SALE values: 0


### Findings
- The `BARCODE` column has notable missing values (e.g., ~5,762).
- `FINAL_QUANTITY` and `FINAL_SALE` are stored as `object`, indicating they contain strings like `"zero"` or `" "`.
- No other columns show null values, but some might contain placeholders (like a single space) instead of true nulls.

In [9]:
# # Replacing blank values in Barcode column with a string value
df['BARCODE'] = df['BARCODE'].fillna("NO_BARCODE_AVAILABLE")
print("Missing barcode values after imputation:", df['BARCODE'].isna().sum())

Missing barcode values after imputation: 0


This shows that there is now no blank values in BARCODE column. And this is important because BARCODE is a foreign key and many databases don't allow NULL values in foreign columns, which could lead to errors when inserting this data into a database.

In [10]:
# Finding number of rows in final sale with a blank
blank_space_count = (df["FINAL_SALE"].str.strip() == "").sum()
print("Number of rows with only whitespace in FINAL_SALE:", blank_space_count)

Number of rows with only whitespace in FINAL_SALE: 12500


In [11]:
# Counting the size of each group
group_sizes = df.groupby(["RECEIPT_ID", "BARCODE"]).size().reset_index(name="count")

print("Distribution of the number of duplicates per (RECEIPT_ID, BARCODE):")
print(group_sizes["count"].value_counts().sort_index())

# This shows how many groups have 1 row, 2 rows, 3 rows, etc.
# 2 is by far the most common, that suggests the "pair" scenario.

Distribution of the number of duplicates per (RECEIPT_ID, BARCODE):
count
2     24615
4       163
6        11
8         5
12        1
Name: count, dtype: int64


##### This shows how many groups have 1 row, 2 rows, 3 rows, etc.
##### 2 is by far the most common, that suggests the "pair" scenario.

In [12]:
# For each (RECEIPT_ID, BARCODE) group, let's check the number of unique values for key columns
check_cols = ["STORE_NAME", "USER_ID", "PURCHASE_DATE", "SCAN_DATE", "FINAL_QUANTITY", "FINAL_SALE"]
agg_dict = {col: lambda x: x.nunique(dropna=False) for col in check_cols}

duplicate_analysis = (
    df.groupby(["RECEIPT_ID", "BARCODE"])
      .agg(agg_dict)
      .reset_index()
)

In [13]:
# Now we can see how many groups have, for example, more than 1 unique value for other columns.
for col in check_cols:
    num_mismatched = (duplicate_analysis[col] > 1).sum()
    print(f"Groups with more than one unique '{col}': {num_mismatched}")

Groups with more than one unique 'STORE_NAME': 0
Groups with more than one unique 'USER_ID': 0
Groups with more than one unique 'PURCHASE_DATE': 0
Groups with more than one unique 'SCAN_DATE': 0
Groups with more than one unique 'FINAL_QUANTITY': 12489
Groups with more than one unique 'FINAL_SALE': 12486


In [14]:
# Now let's see how many unique FINAL_SALE values each group has
sale_unique_count = (
    df.groupby(["RECEIPT_ID", "BARCODE"])["FINAL_SALE"]
      .nunique(dropna=False)
      .reset_index(name="unique_final_sale_count")
)

print("Distribution of the number of unique FINAL_SALE values:")
print(sale_unique_count["unique_final_sale_count"].value_counts().sort_index())

Distribution of the number of unique FINAL_SALE values:
unique_final_sale_count
1    12309
2    12433
3       50
4        3
Name: count, dtype: int64


## What the Distributions Mean

### Unique Final Sale Values (`unique_final_sale_count`)

- **1 unique value**  
  For that ({RECEIPT_ID}, {BARCODE}) group, **all rows** share the same `FINAL_SALE`.
  - **Example**: A group has 2 rows, but both rows have `FINAL_SALE = "3.99"`. That’s just 1 unique final sale value (i.e., `"3.99"`).
  - Another possibility is the group has only **1 row** in total, so naturally there’s only one final sale value.

- **2 unique values**  
  This typically indicates duplicates. One row might have a blank or space-only `FINAL_SALE`, and the other row has a numeric sale amount. Or they could have two different numeric amounts.
  - **Example**:  
    - Row 1: `FINAL_SALE = " "` (blank)  
    - Row 2: `FINAL_SALE = "2.49"`  
    That yields 2 unique final sale values in total.

- **3 or 4 unique values**  
  These are more unusual. It means the group has **3+ rows**, each with different `FINAL_SALE` values, or a mix of numeric and blank placeholders.
  - **Example**:  
    - Row 1: `" "`  
    - Row 2: `"3.99"`  
    - Row 3: `"5.49"`  
    => 3 unique final sale values.

## Similarly we can find for FINAL_QUANTITY as well 

In [15]:
# Let's also see how many unique FINAL_QUANTITY values each group has since Groups with more than one unique 'FINAL_QUANTITY' is 11066
quantity_unique_count = (
    df.groupby(["RECEIPT_ID", "BARCODE"])["FINAL_QUANTITY"]
      .nunique(dropna=False)
      .reset_index(name="unique_final_quantity_count")
)
print("Distribution of the number of unique FINAL_QUANTITY values:")
print(quantity_unique_count["unique_final_quantity_count"].value_counts().sort_index())

Distribution of the number of unique FINAL_QUANTITY values:
unique_final_quantity_count
1    12306
2    12486
3        3
Name: count, dtype: int64


In [16]:
# Merge group_sizes and sale_unique_count so we can filter them together
analysis_df = pd.merge(group_sizes, sale_unique_count, on=["RECEIPT_ID", "BARCODE"], how="left")

# Filter to only the pairs with 2 rows and 2 unique FINAL_SALE values
ideal_pairs = analysis_df[(analysis_df["count"] == 2) & (analysis_df["unique_final_sale_count"] == 2)]

print("Number of (RECEIPT_ID, BARCODE) pairs with exactly 2 rows & 2 unique FINAL_SALE values:", 
      ideal_pairs.shape[0])

Number of (RECEIPT_ID, BARCODE) pairs with exactly 2 rows & 2 unique FINAL_SALE values: 12306


In [17]:
# Merge group_sizes and sale_unique_count so we can filter them together
quantity_analysis_df = pd.merge(group_sizes, quantity_unique_count, on=["RECEIPT_ID", "BARCODE"], how="left")

# Filter to only the pairs with 2 rows and 2 unique FINAL_SALE values
ideal_pairs = quantity_analysis_df[(quantity_analysis_df["count"] == 2) & (quantity_analysis_df["unique_final_quantity_count"] == 2)]

print("Number of (RECEIPT_ID, BARCODE) pairs with exactly 2 rows & 2 unique FINAL_QUANTITY values:", 
      ideal_pairs.shape[0])

Number of (RECEIPT_ID, BARCODE) pairs with exactly 2 rows & 2 unique FINAL_QUANTITY values: 12309


In [18]:
# Drop rows where 'FINAL_QUANTITY' is zero and 'FINAL_SALE' is blank
df_transaction1 = df[df['FINAL_QUANTITY'] != 'zero']
df_transaction1.shape

(37500, 8)

In [19]:
df_transaction2 = df_transaction1[df_transaction1['FINAL_SALE'] != ' ']
transaction_df = df_transaction2.copy()

In [20]:
transaction_df.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
25000,7b3ec72d-9d30-40b8-b185-0bfb638942a9,2024-08-20,2024-08-20 11:17:29.633 Z,DOLLAR GENERAL STORE,60fc1e6deb7585430ff52ee7,745527114884.0,1.00,1.65
25001,04869b68-29e3-4e8d-9bdb-950046fc3473,2024-08-05,2024-08-09 16:06:00.570 Z,DOLLAR GENERAL STORE,654cf234a225ea102b81072e,745527114884.0,1.00,1.65
25002,f1a96308-24a5-46a8-8d8c-285cf9dce1ba,2024-09-03,2024-09-03 11:28:25.264 Z,WALMART,63c1cb6d3d310dceeac55487,37000828761.0,1.00,28.22
25003,7ee1798e-fd2e-4278-838b-f417fdcafe08,2024-08-30,2024-09-04 12:53:31.478 Z,DOLLAR GENERAL STORE,65c29b137050d0a6206cd24f,12000504051.0,1.00,5.25
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125 Z,TARGET,61a58ac49c135b462ccddd1c,24000393429.0,1.00,2.59


In [21]:
# df_transaction2 = df_transaction2.copy()

# df_transaction2['PURCHASE_DATE'] = pd.to_datetime(
#     df_transaction2['PURCHASE_DATE'], errors='coerce'
# ).dt.tz_localize(None)

# df_transaction2['SCAN_DATE'] = pd.to_datetime(
#     df_transaction2['SCAN_DATE'], errors='coerce'
# ).dt.tz_localize(None)

To ensure data integrity, we need to confirm that all Receipt ID and Barcode combinations remain intact after cleaning.

In [22]:
# 1) Extract unique (RECEIPT_ID, BARCODE) pairs from the original data
unique_orig = df[['RECEIPT_ID', 'BARCODE']].drop_duplicates()

# 2) Extract unique (RECEIPT_ID, BARCODE) pairs from the cleaned data
unique_clean = transaction_df[['RECEIPT_ID', 'BARCODE']].drop_duplicates()

# 3) Merge the two DataFrames using a left join, and track matches with an indicator column
merged = pd.merge(unique_orig, 
                  unique_clean, 
                  on=['RECEIPT_ID', 'BARCODE'], 
                  how='left', 
                  indicator=True)

# 4) Any row with '_merge' == 'left_only' indicates a (RECEIPT_ID, BARCODE) pair
#    from the original data that's missing in the cleaned data
missing_rows = merged[merged['_merge'] == 'left_only']

if missing_rows.empty:
    print("All unique (RECEIPT_ID, BARCODE) combos from the original data are present in the cleaned data.")
else:
    print("Some combos from the original data are missing in the cleaned data. Details:")
    display(missing_rows)

All unique (RECEIPT_ID, BARCODE) combos from the original data are present in the cleaned data.


In [23]:
# df_transaction2.to_csv("TRANSACTION_CLEANED.csv", index=False)

### Standardizing Barcodes for Consistency
To ensure accurate mapping between transactions and product details, we need to standardize barcodes across both datasets. Differences in data types, missing leading zeros, or inconsistent formatting may cause mismatches.

In [24]:
products_df = pd.read_csv("PRODUCTS_TAKEHOME.csv")

# Convert barcode column to string format
transaction_df['BARCODE'] = transaction_df['BARCODE'].astype(str)
products_df['BARCODE'] = products_df['BARCODE'].astype(str)

In [25]:
transaction_df.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
25000,7b3ec72d-9d30-40b8-b185-0bfb638942a9,2024-08-20,2024-08-20 11:17:29.633 Z,DOLLAR GENERAL STORE,60fc1e6deb7585430ff52ee7,745527114884.0,1.00,1.65
25001,04869b68-29e3-4e8d-9bdb-950046fc3473,2024-08-05,2024-08-09 16:06:00.570 Z,DOLLAR GENERAL STORE,654cf234a225ea102b81072e,745527114884.0,1.00,1.65
25002,f1a96308-24a5-46a8-8d8c-285cf9dce1ba,2024-09-03,2024-09-03 11:28:25.264 Z,WALMART,63c1cb6d3d310dceeac55487,37000828761.0,1.00,28.22
25003,7ee1798e-fd2e-4278-838b-f417fdcafe08,2024-08-30,2024-09-04 12:53:31.478 Z,DOLLAR GENERAL STORE,65c29b137050d0a6206cd24f,12000504051.0,1.00,5.25
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125 Z,TARGET,61a58ac49c135b462ccddd1c,24000393429.0,1.00,2.59


In [26]:
products_df.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820.0
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,23278011028.0
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225.0
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,35000466815.0
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,806810850459.0


In [27]:
# Check barcode lengths in both datasets
print(transaction_df['BARCODE'].str.len().value_counts())
print(products_df['BARCODE'].str.len().value_counts())

BARCODE
13    16344
14     5271
20     2881
12      158
7       125
9        75
10       75
15       43
8        20
4         4
11        2
6         2
Name: count, dtype: int64
BARCODE
14    502091
13    296376
15     30144
12      7591
3       4025
10      1907
11      1762
9        950
8        570
7         83
16        44
6          8
5          1
Name: count, dtype: int64


In [40]:
# Ensure all barcodes are 13 characters long by padding with leading zeros
transaction_df['BARCODE'] = transaction_df['BARCODE'].str.zfill(13)
products_df['BARCODE'] = products_df['BARCODE'].str.zfill(13)

In [41]:
# Strip spaces and remove any unexpected characters
transaction_df['BARCODE'] = transaction_df['BARCODE'].str.strip()
products_df['BARCODE'] = products_df['BARCODE'].str.strip()

In [42]:
# Check barcode lengths in both datasets
print(transaction_df['BARCODE'].str.len().value_counts())
print(products_df['BARCODE'].str.len().value_counts())

BARCODE
13    16805
14     5271
20     2881
15       43
Name: count, dtype: int64
BARCODE
14    815364
15     30144
16        44
Name: count, dtype: int64


In [43]:
# Find barcodes in transactions that are missing in products
missing_barcodes = df_transaction2[~df_transaction2['BARCODE'].isin(products_df['BARCODE'])]['BARCODE'].unique()

print(f"Number of barcodes in transactions not found in products: {len(missing_barcodes)}")
if len(missing_barcodes) > 0:
    print("Sample missing barcodes:", missing_barcodes[:10])

Number of barcodes in transactions not found in products: 2
Sample missing barcodes: [nan 'NO_BARCODE_AVAILABLE']


In [44]:
# Find barcodes in products that are missing in transactions
missing_product_barcodes = products_df[~products_df['BARCODE'].isin(df_transaction2['BARCODE'])]['BARCODE'].unique()

print(f"Number of barcodes in products not found in transactions: {len(missing_product_barcodes)}")
if len(missing_product_barcodes) > 0:
    print("Sample missing barcodes:", missing_product_barcodes[:10])

Number of barcodes in products not found in transactions: 841343
Sample missing barcodes: ['796494407820.0' '023278011028.0' '461817824225.0' '035000466815.0'
 '806810850459.0' '662658519183.0' '617737584217.0' '7501839106268.0'
 '075450128345.0' '00000000000nan']


## **Issue: Barcode Length Inconsistencies**

During our analysis, we found that barcodes in the transactions and products datasets have **inconsistent lengths**, leading to mismatches when trying to link them. The **maximum barcode length observed is 14 characters**, meaning some barcodes might be missing leading zeros or have formatting inconsistencies.

### **Identified Problems**
- Barcodes in the transactions dataset vary in length.
- Barcodes in the products dataset also have varying lengths.
- Some barcodes may be missing due to these inconsistencies, affecting data integrity.

## **Solution: Standardizing Barcode Length**
To resolve this issue, we will:
1. **Convert all barcodes to strings** to maintain consistency across both datasets.
2. **Standardize all barcodes to 14 characters** by **padding shorter barcodes with leading zeros**.
3. **Revalidate missing barcodes after standardization** to check if mismatches are resolved.

By ensuring all barcodes follow the same format, we can correctly link transactions to products and improve data accuracy.

In [45]:
# Padding barcodes with leading zeros to ensure they are all 14 characters long since 14 was the longest in our observation
transaction_df['BARCODE'] = transaction_df['BARCODE'].str.zfill(14)
products_df['BARCODE'] = products_df['BARCODE'].str.zfill(14)

In [48]:
# Counting barcodes in transactions that are found in the products dataset
matched_barcodes = transaction_df[transaction_df['BARCODE'].isin(products_df['BARCODE'])]['BARCODE'].nunique()
total_barcodes = transaction_df['BARCODE'].nunique()

# Calculate the percentage of matched barcodes
match_percentage = (matched_barcodes / total_barcodes) * 100

print(f"Total unique barcodes in transactions: {total_barcodes}")
print(f"Number of matched barcodes in products: {matched_barcodes}")
print(f"Percentage of matched barcodes: {match_percentage:.2f}%")

Total unique barcodes in transactions: 11028
Number of matched barcodes in products: 6562
Percentage of matched barcodes: 59.50%


Only 59.50% of transaction barcodes match with the products dataset, indicating possible formatting inconsistencies or missing product records. This mismatch could impact product-level analysis

# Now let's check matching with the USER data

In [49]:
df_users = pd.read_csv("Updated_Datasets/USER_CLEANED.csv")

missing_ids_raw = set(transaction_df['USER_ID']) - set(df_users['ID'])

# Calculating the total unique USER_IDs in transactions
total_trans_ids = transaction_df['USER_ID'].nunique()

# Determining the number of matched USER_IDs by subtracting missing from total
matched_ids_raw = total_trans_ids - len(missing_ids_raw)

# Percentages
missing_percentage = (len(missing_ids_raw) / total_trans_ids) * 100
matched_percentage = (matched_ids_raw / total_trans_ids) * 100

print("Total unique USER_IDs in transactions:", total_trans_ids)
print("Number of USER_IDs not present in the users table:", len(missing_ids_raw))
print("Percentage of missing USER_IDs: {:.2f}%".format(missing_percentage))
print("Number of matched USER_IDs:", matched_ids_raw)
print("Percentage of matched USER_IDs: {:.2f}%".format(matched_percentage))

Total unique USER_IDs in transactions: 17694
Number of USER_IDs not present in the users table: 17603
Percentage of missing USER_IDs: 99.49%
Number of matched USER_IDs: 91
Percentage of matched USER_IDs: 0.51%


USER_ID Matching Analysis Summary

Only **0.51% of USER_IDs from transactions match with the users database**, indicating a substantial data linkage issue likely due to inconsistencies in USER_ID recording or a mismatch in the datasets' scopes.

In [50]:
products_df.shape

(845552, 7)

In [51]:
transaction_df.shape

(25000, 8)

In [52]:
products_df.dtypes

CATEGORY_1      object
CATEGORY_2      object
CATEGORY_3      object
CATEGORY_4      object
MANUFACTURER    object
BRAND           object
BARCODE         object
dtype: object

In [53]:
transaction_df.to_csv("Updated_Datasets/TRANSACTION_CLEANED.csv", index=False)
products_df.to_csv("Updated_Datasets/PRODUCTS_CLEANED.csv", index=False)